In [56]:
from tensorflow.keras.models import load_model

In [57]:
import tensorflow as tf

class EnforceScoreConstraintLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        raw_scores, best_of_unscaled = inputs # raw_scores: (batch, 2), best_of_unscaled: (batch, 1)

        # Ensure best_of_unscaled is float for calculations
        best_of_unscaled = tf.cast(best_of_unscaled, tf.float32)

        # Calculate S_win (sets needed to win) for each sample in the batch
        S_win = tf.math.ceil(best_of_unscaled / 2.0) # Shape: (batch, 1)
        max_loser_sets = S_win - 1.0 # Shape: (batch, 1)

        o1 = raw_scores[:, 0:1] # Shape: (batch, 1), raw prediction for P1
        o2 = raw_scores[:, 1:2] # Shape: (batch, 1), raw prediction for P2

        # Determine the winner based on raw predictions o1, o2
        # p1_wins_condition is true if o1 >= o2. P1 wins ties.
        p1_wins_condition = o1 >= o2
        p1_wins = tf.cast(p1_wins_condition, tf.float32)
        # p2_wins is true if o1 < o2.
        p2_wins = tf.cast(tf.logical_not(p1_wins_condition), tf.float32)

        # Predict loser's score: use relu on raw score (to make it non-negative)
        # then clip it by max_loser_sets.
        o1_pos_for_loser_score = tf.nn.relu(o1) # P1's potential score if P1 loses
        o2_pos_for_loser_score = tf.nn.relu(o2) # P2's potential score if P2 loses
        
        # Candidate score for P1 if P1 loses (non-negative and clipped)
        s1_as_loser = tf.clip_by_value(o1_pos_for_loser_score, 0.0, max_loser_sets)
        # Candidate score for P2 if P2 loses (non-negative and clipped)
        s2_as_loser = tf.clip_by_value(o2_pos_for_loser_score, 0.0, max_loser_sets)

        # Final scores: winner gets S_win, loser gets their candidate score
        final_s1 = p1_wins * S_win        + p2_wins * s1_as_loser
        final_s2 = p1_wins * s2_as_loser  + p2_wins * S_win
        
        # Concatenate to form the output tensor (batch_size, 2)
        constrained_scores = tf.concat([final_s1, final_s2], axis=1)
        return constrained_scores

    def get_config(self):
        # Required for saving/loading models with custom layers
        config = super().get_config()
        return config


In [64]:
model_path = "trained_tennis_model2.keras" # Passen Sie diese Variable beliebig an
loaded_model = load_model(model_path, custom_objects={"EnforceScoreConstraintLayer": EnforceScoreConstraintLayer})

In [59]:
def predict(X):
    y_pred = loaded_model.predict(X)
    result = []
    for entry in y_pred:
        # entry is an array, so round each value
        result.append([round(float(x)) for x in entry])
    return result

# NICHT DIE STRUKTUR VERÄNDERN
In der folgenden Zelle muss die Struktur exakt so erhalten bleiben, wie sie im Kommentar aufgelistet ist. Es dürfen lediglich die Werte ersetzt werden, aber es müssen immer alle Werte in exakt dieser Reihenfolge angegeben werden. Sollten Sie die Werte noch bearbeiten müssen, tun Sie dies bitte in der Zelle darunter. Es dürfen keine weiteren Werte wie Betting Rates in die Evaluation einfließen.

In [60]:
# ATP, Location, Tournament, Date, Series, Court, Surface, Round, Best of, Winner, Loser, WRank, LRank, WPts, LPts
X = [1, "Brisbane", "Brisbane International", "31.12.2023", "ATP250", "Outdoor", "Hard", "1st Round", 3, "Popyrin A.", "O Connell C.", 40, 68, 1084, 780]

In [61]:
X = [1, "Paris", "Roland Garros", "26.05.2024", "Grand Slam", "Outdoor", "Clay", "Quarterfinals", 5, "Alcaraz C.", "Djokovic N.", 2, 1, 8900, 11200]

In [62]:
def prepare_data(X):
    import numpy as np
    x_new = [X[11], X[12], X[13], X[14], X[8], X[6], X[7], X[4], X[5]]
    # Directly embedded preprocessing info (copy from your preprocessing_info.json)
    preprocessing_info = {
        "numerical_features": [
            "p1_rank",
            "p2_rank",
            "p1_pts",
            "p2_pts",
            "best_of"
        ],
        "categorical_features": [
            "surface",
            "round",
            "series_cat",
            "court_type"
        ],
        "scaler_means": [
            65.51626016260163,
            65.51626016260163,
            2081.691056910569,
            2081.691056910569,
            5.0
        ],
        "scaler_scales": [
            92.05171388024256,
            92.05171388024256,
            2270.618780066695,
            2270.618780066695,
            1.0
        ],
        "onehot_categories": {
            "surface": [
                "Clay"
            ],
            "round": [
                "1st Round",
                "2nd Round",
                "3rd Round",
                "4th Round",
                "Quarterfinals",
                "Semifinals",
                "The Final"
            ],
            "series_cat": [
                "Grand Slam"
            ],
            "court_type": [
                "Outdoor"
            ]
        },
        "feature_order": [
            "p1_rank",
            "p2_rank",
            "p1_pts",
            "p2_pts",
            "best_of",
            "surface_Clay",
            "round_1st Round",
            "round_2nd Round",
            "round_3rd Round",
            "round_4th Round",
            "round_Quarterfinals",
            "round_Semifinals",
            "round_The Final",
            "series_cat_Grand Slam",
            "court_type_Outdoor"
        ]
    }

    numerical_features = preprocessing_info['numerical_features']
    categorical_features = preprocessing_info['categorical_features']
    scaler_means = np.array(preprocessing_info['scaler_means'])
    scaler_scales = np.array(preprocessing_info['scaler_scales'])
    onehot_categories = preprocessing_info['onehot_categories']

    # Map input X (list) to feature dict
    feature_dict = dict(zip(
        ['p1_rank', 'p2_rank', 'p1_pts', 'p2_pts', 'best_of', 'surface', 'round', 'series_cat', 'court_type'],
        x_new[:9]
    ))

    # Prepare numerical features
    num_vals = []
    for i, col in enumerate(numerical_features):
        val = float(feature_dict[col])
        val = (val - scaler_means[i]) / scaler_scales[i]
        num_vals.append(val)

    # Prepare categorical features (one-hot)
    cat_vals = []
    for col in categorical_features:
        cats = onehot_categories[col]
        val = feature_dict[col]
        onehot = [1.0 if val == c else 0.0 for c in cats]
        cat_vals.extend(onehot)

    # Combine all features
    X_prepared = np.array(num_vals + cat_vals, dtype=np.float32).reshape(1, -1)
    return X_prepared

In [65]:
# Hier wird ausgewertet. Es wird hier die Anzahl an gewonnen Sätzen entsprechend der Reihenfolge der Spieler erwartet.
# Bsp: Es spielen Spieler A und Spieler B und diese werden auch in dieser Reihenfolge angegeben.
# Spieler B gewinnt mit 2:1. Er ist aber als zweites gelistet. Die erwartete Ausgabe soll also [1,2] lauten.
prepared_X = prepare_data(X)
prediction = predict(prepared_X)
print(prediction)

ValueError: Layer "functional" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(1, 15) dtype=float32>]